# Testing data prepared to predict

Strategy: 
- collect an arbitrary data period that contains (begin, end).
- the last value is end. a new_begin is calculated in data_manner.__add_period
- prepare data with build_test(). this automatically removes the last input_window_size from the data to build x, which is the array we will input to predict
- predicts
- get an array of predictions y_hat from new_begin+7 to end
- get the last (end-begin) values from y_hat. this is the forecast from begin to end

In [1]:
configure_json = open('../doc/configure.json', 'r')

import json
#print(json.dumps(json.load(configure_json), indent=4))

In [2]:
import sys
sys.path.append("../src")

import configs_manner

# print("Model infos: \n", configs_manner.model_infos)
# print("\n")
# print("Models path: \n", configs_manner.model_path)
# print("\n")
# print("Data path: \n", configs_manner.data_path)

In [3]:
# import the data_manner.py file. (taking into account that you are in src/ path)
import data_manner
import predictor_manner
from importlib import reload
reload(data_manner)
reload(predictor_manner)

import numpy as np
import datetime

np.set_printoptions(precision=4, suppress=True)

# creating the DataConstructor instance
data_constructor_train = data_manner.DataConstructor()
data_constructor = data_manner.DataConstructor(True)

model_id = "2f36b1b8-c0d1-11ec-9f78-132ad413b391"
# specif code to the remote repository data.
repo = "p971074907"
# country and state acronym splitted by a ":"
path = "brl:am"
# columns (or features) to be extracted from the database, each one splitted by a ":"
feature = "date:newDeaths:"
# start date for the data request.
begin = "2022-04-10"
# finish date for the data request.
end = "2022-06-05"

DATE_FORMAT = "%Y-%m-%d"

# # collect data from the remote repository.
# collected_data_train = data_constructor_train.collect_dataframe(path, repo, feature, begin, end)
# collected_data = data_constructor.collect_dataframe(path, repo, feature, begin, end)

# print("DATA FOR TRAINING", "period:", begin, "-", end)
# # print(collected_data_train)
# # print("\n")
# print("RAW DATA", "period after add_period:", data_constructor.new_first_day.strftime(DATE_FORMAT), "-", end)
# # print("DATA FOR PREDICTING", "period after removing ma buffer:", (data_constructor.new_first_day+datetime.timedelta(days=7)).strftime(DATE_FORMAT), "-", data_constructor.new_last_day.strftime(DATE_FORMAT))
# print(data_constructor.processed_data_raw)
# print(data_constructor.processed_data_raw.shape[0], "days")
# print("NEW DATA", "period after extrapolate check:", data_constructor.new_first_day.strftime(DATE_FORMAT), "-", data_constructor.new_last_day.strftime(DATE_FORMAT))
# # print("NEW DATA", "period after check for extrapolate:", data_constructor.new_first_day.strftime(DATE_FORMAT), "-", data_constructor.new_last_day.strftime(DATE_FORMAT))
# # print("Processed data", data_constructor.processed_data_new.shape[0])
# print(data_constructor.processed_data_new)
# print(data_constructor.processed_data_new.shape[0], "days")

predictor = predictor_manner.PredictorConstructor("2f36b1b8-c0d1-11ec-9f78-132ad413b391", path, repo, feature, begin, end)
y_hat = predictor.predict()

# # print("\n")
# print("DATA X prepared by predictor:", data_constructor.new_first_day.strftime(DATE_FORMAT), data_constructor.new_last_day.strftime(DATE_FORMAT))
# print(predictor.data_X.flatten())
# print(predictor.data_X.flatten().shape)
# # print("\n")
# print("RAW PREDICTION using data_X:", data_constructor.new_first_day.strftime(DATE_FORMAT), data_constructor.new_last_day.strftime(DATE_FORMAT))
# print(predictor.raw_y_hat)
# print(predictor.raw_y_hat.flatten().shape)
# print("OFFSET PREDICTION using data_X:", (data_constructor.new_last_day - datetime.timedelta(days=y_hat.shape[0])).strftime(DATE_FORMAT), data_constructor.new_last_day.strftime(DATE_FORMAT))
# print(y_hat)
# print(y_hat.flatten().shape)

In [4]:
y_hat_weboutput = predictor.predictions_to_weboutput(y_hat)
y_hat_weboutput

[{'date': '2022-04-10', 'prediction': '6.649837'},
 {'date': '2022-04-11', 'prediction': '6.5123024'},
 {'date': '2022-04-12', 'prediction': '3.4979966'},
 {'date': '2022-04-13', 'prediction': '3.9140964'},
 {'date': '2022-04-14', 'prediction': '4.958813'},
 {'date': '2022-04-15', 'prediction': '6.0589323'},
 {'date': '2022-04-16', 'prediction': '6.746418'},
 {'date': '2022-04-17', 'prediction': '7.0510798'},
 {'date': '2022-04-18', 'prediction': '7.048666'},
 {'date': '2022-04-19', 'prediction': '3.2842014'},
 {'date': '2022-04-20', 'prediction': '3.8004494'},
 {'date': '2022-04-21', 'prediction': '4.8184533'},
 {'date': '2022-04-22', 'prediction': '5.9207397'},
 {'date': '2022-04-23', 'prediction': '6.6829853'},
 {'date': '2022-04-24', 'prediction': '7.0981674'},
 {'date': '2022-04-25', 'prediction': '7.22366'},
 {'date': '2022-04-26', 'prediction': '3.640121'},
 {'date': '2022-04-27', 'prediction': '4.143515'},
 {'date': '2022-04-28', 'prediction': '5.167095'},
 {'date': '2022-04-29

In [5]:
y_hat_weboutput = predictor.predictions_to_weboutput(y_hat)
y_hat_weboutput

[{'date': '2022-04-10', 'prediction': '6.649837'},
 {'date': '2022-04-11', 'prediction': '6.5123024'},
 {'date': '2022-04-12', 'prediction': '3.4979966'},
 {'date': '2022-04-13', 'prediction': '3.9140964'},
 {'date': '2022-04-14', 'prediction': '4.958813'},
 {'date': '2022-04-15', 'prediction': '6.0589323'},
 {'date': '2022-04-16', 'prediction': '6.746418'},
 {'date': '2022-04-17', 'prediction': '7.0510798'},
 {'date': '2022-04-18', 'prediction': '7.048666'},
 {'date': '2022-04-19', 'prediction': '3.2842014'},
 {'date': '2022-04-20', 'prediction': '3.8004494'},
 {'date': '2022-04-21', 'prediction': '4.8184533'},
 {'date': '2022-04-22', 'prediction': '5.9207397'},
 {'date': '2022-04-23', 'prediction': '6.6829853'},
 {'date': '2022-04-24', 'prediction': '7.0981674'},
 {'date': '2022-04-25', 'prediction': '7.22366'},
 {'date': '2022-04-26', 'prediction': '3.640121'},
 {'date': '2022-04-27', 'prediction': '4.143515'},
 {'date': '2022-04-28', 'prediction': '5.167095'},
 {'date': '2022-04-29

In [126]:
y_hat_weboutput = predictor.predictions_to_weboutput(y_hat)
y_hat_weboutput

[{'date': '2022-04-15', 'prediction': '6.7776017'},
 {'date': '2022-04-16', 'prediction': '7.022166'},
 {'date': '2022-04-17', 'prediction': '6.9675164'},
 {'date': '2022-04-18', 'prediction': '3.2842014'},
 {'date': '2022-04-19', 'prediction': '3.8004494'},
 {'date': '2022-04-20', 'prediction': '4.8184533'},
 {'date': '2022-04-21', 'prediction': '5.9207397'},
 {'date': '2022-04-22', 'prediction': '6.6829853'},
 {'date': '2022-04-23', 'prediction': '7.0981674'},
 {'date': '2022-04-24', 'prediction': '7.22366'},
 {'date': '2022-04-25', 'prediction': '3.640121'},
 {'date': '2022-04-26', 'prediction': '4.143515'},
 {'date': '2022-04-27', 'prediction': '5.167095'},
 {'date': '2022-04-28', 'prediction': '6.1581006'},
 {'date': '2022-04-29', 'prediction': '6.491473'},
 {'date': '2022-04-30', 'prediction': '6.3761473'},
 {'date': '2022-05-01', 'prediction': '5.9976587'},
 {'date': '2022-05-02', 'prediction': '3.0282254'},
 {'date': '2022-05-03', 'prediction': '3.814426'},
 {'date': '2022-05-0

In [62]:
import numpy as np
import matplotlib.pyplot as plt

def uniqueish_color():
    """There're better ways to generate unique colors, but this isn't awful."""
    return plt.cm.gist_ncar(np.random.random())

def plot_weeks(x_predict):
    size = x_predict.x.flatten().shape[0]
    x_axis = list(range(size))

    fig, ax = plt.subplots()

    xy = np.vstack((x_axis, x_predict.x.flatten()))

    for i in range(xy.shape[1]//7):
        if i == 0:
            begin = 0
            end = (i+1)*7
        else:
            begin = (i*7)-1
            end = (i+1)*7
        ax.plot(xy[0][begin:end], xy[1][begin:end], color=uniqueish_color())
    plt.show()